Tải dữ liệu và hiển thị thông tin cơ bản

In [ ]:
import pandas as pd

# Đọc dữ liệu từ file CSV
file_path = "dulieuxettuyendaihoc.csv"  # Đảm bảo đường dẫn file đúng
df = pd.read_csv(file_path)

# Hiển thị thông tin về dữ liệu
print("Thông tin về dataset:")
print(df.info())

# Hiển thị 10 dòng đầu tiên
print("\n10 dòng đầu:")
print(df.head(10))

# Hiển thị 10 dòng cuối
print("\n10 dòng cuối:")
print(df.tail(10))


1. Phân loại dữ liệu định tính và định lượng
Chúng ta sẽ chia dữ liệu thành hai nhóm:

-Dữ liệu định tính (Categorical Data):

GT (Giới tính), DT (Dân tộc), KV (Khu vực), KT (Khối thi)

-Dữ liệu định lượng (Numerical Data):

Các điểm số trung bình (T1, L1, H1, ... đến N6)

Các điểm thi đại học (DH1, DH2, DH3)


In [ ]:
qualitative_vars = ["GT", "DT", "KV", "KT"]
quantitative_vars = [col for col in df.columns if col not in qualitative_vars]

print("Dữ liệu định tính:", qualitative_vars)
print("Dữ liệu định lượng:", quantitative_vars)


2. Định nghĩa thang đo phù hợp cho từng biến số
Danh mục (Nominal Scale): GT, DT, KV, KT

Thứ tự (Ordinal Scale): XL1, XL2, XL3 (xếp loại học lực)

Khoảng (Interval Scale) & Tỉ lệ (Ratio Scale): Các điểm số (T1, DH1, ...)

3. Đọc dữ liệu và hiển thị 10 dòng đầu và cuối

In [ ]:
print("10 dòng đầu:")
print(df.head(10))

print("\n10 dòng cuối:")
print(df.tail(10))


4. Xử lý dữ liệu thiếu cho cột DT
Chúng ta sẽ thay thế giá trị thiếu của DT bằng 0.

In [ ]:
df["DT"].fillna(0, inplace=True)


Sau đó kiểm tra lại:

In [ ]:
print(df["DT"].isnull().sum())  # Xem còn giá trị thiếu không


5. Xử lý dữ liệu thiếu cho biến T1 bằng phương pháp Mean

In [ ]:
df["T1"].fillna(df["T1"].mean(), inplace=True)


Kiểm tra lại dữ liệu:

In [ ]:
print(df["T1"].isnull().sum())  # Phải ra kết quả 0


6. Xử lý dữ liệu thiếu cho các biến điểm số khác

In [ ]:
score_columns = ["T1", "L1", "H1", "S1", "V1", "X1", "D1", "N1",
                 "T2", "L2", "H2", "S2", "V2", "X2", "D2", "N2",
                 "T6", "L6", "H6", "S6", "V6", "X6", "D6", "N6",
                 "DH1", "DH2", "DH3"]

for col in score_columns:
    df[col].fillna(df[col].mean(), inplace=True)
print(df.isnull().sum())  # Phải không còn giá trị thiếu


7. Tạo biến TBM1, TBM2, TBM3 (Trung bình môn các năm)

In [ ]:
def calculate_TBM(row, year):
    return (row[f"T{year}"]*2 + row[f"L{year}"] + row[f"H{year}"] + row[f"S{year}"] +
            row[f"V{year}"]*2 + row[f"X{year}"] + row[f"D{year}"] + row[f"N{year}"]) / 10

df["TBM1"] = df.apply(lambda row: calculate_TBM(row, 1), axis=1)
df["TBM2"] = df.apply(lambda row: calculate_TBM(row, 2), axis=1)
df["TBM3"] = df.apply(lambda row: calculate_TBM(row, 6), axis=1)


8. Tạo biến xếp loại XL1, XL2, XL3

In [ ]:
def classify_grade(score):
    if score < 5.0:
        return "Y"
    elif 5.0 <= score < 6.5:
        return "TB"
    elif 6.5 <= score < 8.0:
        return "K"
    elif 8.0 <= score < 9.0:
        return "G"
    else:
        return "XS"

df["XL1"] = df["TBM1"].apply(classify_grade)
df["XL2"] = df["TBM2"].apply(classify_grade)
df["XL3"] = df["TBM3"].apply(classify_grade)


9. Chuyển điểm TBM sang thang điểm 4 (US_TBM)

In [ ]:
def min_max_scale(x, min_val, max_val):
    return 4 * (x - min_val) / (max_val - min_val)

min_tbm = df[["TBM1", "TBM2", "TBM3"]].min().min()
max_tbm = df[["TBM1", "TBM2", "TBM3"]].max().max()

df["US_TBM1"] = df["TBM1"].apply(lambda x: min_max_scale(x, min_tbm, max_tbm))
df["US_TBM2"] = df["TBM2"].apply(lambda x: min_max_scale(x, min_tbm, max_tbm))
df["US_TBM3"] = df["TBM3"].apply(lambda x: min_max_scale(x, min_tbm, max_tbm))


10. Tạo biến KQXT (Kết quả xét tuyển đậu/rớt)

In [ ]:
def determine_result(row):
    if row["KT"] in ["A", "A1"]:
        score = (row["DH1"]*2 + row["DH2"] + row["DH3"]) / 4
    elif row["KT"] == "B":
        score = (row["DH1"] + row["DH2"]*2 + row["DH3"]) / 4
    else:
        score = (row["DH1"] + row["DH2"] + row["DH3"]) / 3

    return 1 if score >= 5.0 else 0

df["KQXT"] = df.apply(determine_result, axis=1)


11. Lưu dữ liệu đã xử lý vào file CSV mới

In [ ]:
df.to_csv("processed_dulieuxettuyendaihoc.csv", index=False)
print("Dữ liệu đã được lưu thành công!")
